In [1]:
import libspn as spn
import tensorflow as tf

import numpy as np
sess = tf.InteractiveSession()

In [2]:
# Inputs
ivs = spn.IVs(num_vars=2, num_vals=2, name="IVs")

# Input mixtures
# Sums-1
s1 = spn.Sums((ivs, [0, 1]), num_sums=2, name="Sums1")
s1.generate_weights([0.4, 0.6, 0.1, 0.9])

# Sums-2
s2 = spn.Sums((ivs, [2, 3]), num_sums=2, name="Sums2")
s2.generate_weights([0.7, 0.3, 0.8, 0.2])

p = spn.Products((s1, [0]), (s2, [0]),
             (s1, [0]), (s2, [1]),
             (s1, [1]), (s2, [1]),
             num_prods=3, name="Comp1-2-3")

# Mixing components
root = spn.Sums((p, [0, 1, 2]), name="Mixture")
root.generate_weights([0.5, 0.2, 0.3])

# Init weights
spn.initialize_weights(root).run()

In [3]:
spn.display_spn_graph(root)

In [4]:
# Feed
values = np.arange(-1, 2)
points = np.array(np.meshgrid(*[values for i in range(2)])).T
feed = points.reshape(-1, points.shape[-1])

# True value
true_values = np.array([[1.0],
                        [0.75],
                        [0.25],
                        [0.31],
                        [0.228],
                        [0.082],
                        [0.69],
                        [0.522],
                        [0.168]], dtype=spn.conf.dtype.as_numpy_dtype)

# True MPE value
true_mpe_values = np.array([[0.216],
                            [0.216],
                            [0.09],
                            [0.14],
                            [0.14],
                            [0.06],
                            [0.216],
                            [0.216],
                            [0.09]], dtype=spn.conf.dtype.as_numpy_dtype)

In [5]:
# Value
value=root.get_value()
log_value=root.get_log_value()

In [6]:
value_array=value.eval(feed_dict={ivs: feed})
value_array_log=np.exp(log_value.eval(feed_dict={ivs: feed}))

print(value_array)
print(true_values)
np.testing.assert_almost_equal(value_array, true_values)
np.testing.assert_almost_equal(value_array_log, true_values)

[[ 1.        ]
 [ 0.75      ]
 [ 0.25000003]
 [ 0.31      ]
 [ 0.22800002]
 [ 0.082     ]
 [ 0.69      ]
 [ 0.52200001]
 [ 0.16800001]]
[[ 1.        ]
 [ 0.75      ]
 [ 0.25      ]
 [ 0.31      ]
 [ 0.228     ]
 [ 0.082     ]
 [ 0.69      ]
 [ 0.52200001]
 [ 0.168     ]]


In [7]:
# MPE Value
mpe_value = root.get_value(spn.InferenceType.MPE)
log_mpe_value = root.get_log_value(spn.InferenceType.MPE)

In [8]:
mpe_value_array=mpe_value.eval(feed_dict={ivs: feed})
mpe_value_array_log=np.exp(log_mpe_value.eval(feed_dict={ivs: feed}))

print(mpe_value_array)
print(true_mpe_values)
np.testing.assert_almost_equal(mpe_value_array, true_mpe_values)
np.testing.assert_almost_equal(mpe_value_array_log, true_mpe_values)

[[ 0.21600001]
 [ 0.21600001]
 [ 0.09      ]
 [ 0.14      ]
 [ 0.14      ]
 [ 0.06      ]
 [ 0.21600001]
 [ 0.21600001]
 [ 0.09      ]]
[[ 0.21600001]
 [ 0.21600001]
 [ 0.09      ]
 [ 0.14      ]
 [ 0.14      ]
 [ 0.06      ]
 [ 0.21600001]
 [ 0.21600001]
 [ 0.09      ]]


In [9]:
# MPE MPE Path
mpe_mpe_path_gen = spn.MPEPath(value_inference_type=spn.InferenceType.MPE, log=False)
mpe_mpe_path_gen.get_mpe_path(root)

#print(mpe_mpe_path_gen.counts[ivs].eval(feed_dict={ivs: feed}))

ValueError: 'out_num_cols' must be larger than the size of the indexed dimension of 'params'

In [ ]:
# MPE Marginal Path
mpe_marginal_path_gen = spn.MPEPath(value_inference_type=spn.InferenceType.MARGINAL, log=False)
mpe_marginal_path_gen.get_mpe_path(root)

#print(mpe_marginal_path_gen.counts[ivs].eval(feed_dict={ivs: feed}))